# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
boston = load_boston()
X = boston.data
y = boston.target

In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, shuffle = False)

In [3]:
'''X_train = X[127:, :]
X_test = X[:127, :]
y_train = y[127:]
y_test = y[:127]'''

'X_train = X[127:, :]\nX_test = X[:127, :]\ny_train = y[127:]\ny_test = y[:127]'

In [4]:
X_test.shape

(127, 13)

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [5]:
base_algorithms_list = []
coefficients_list = []
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
z = y_train

basic_coef = 0.9
for i in range(0, 50):
    
    base_algorithms_list.append(DecisionTreeRegressor(max_depth=5, random_state=42))
    base_algorithms_list[i].fit(X_train, z)
    coefficients_list.append(basic_coef)
    z = -1 * (gbm_predict(X_train) - y_train)

In [6]:
from sklearn.metrics import mean_squared_error
x=(mean_squared_error(y_test, gbm_predict(X_test)))**0.5
print(x)

5.455565103009402


## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [7]:
base_algorithms_list = []
coefficients_list = []
def gbm_predict2(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
z = y_train

basic_coef = 0.9
for i in range(0, 50):
    basic_coef = 0.9 / (1.0 + i)
    base_algorithms_list.append(DecisionTreeRegressor(max_depth=5, random_state=42))
    base_algorithms_list[i].fit(X_train, z)
    coefficients_list.append(basic_coef)
    z = -1 * (gbm_predict(X_train) - y_train)

In [8]:
from sklearn.metrics import mean_squared_error
x=(mean_squared_error(y_test, gbm_predict2(X_test)))**0.5
print(x)

4.812550945781193


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [15]:
%%time
from sklearn.ensemble import GradientBoostingRegressor
qual_est = []
for i in range(10, 500, 10):
    for j in range(2, 30):
        gbr = GradientBoostingRegressor(n_estimators = i, max_depth = j)
        gbr.fit(X_train, y_train)
        mse = mean_squared_error(y_test, gbr.predict(X_test))
        qual_est.append((i, j, mse))
        
print(qual_est)

[(10, 2, 68.69486522661833), (10, 3, 55.86723803846859), (10, 4, 46.08192366281771), (10, 5, 44.04932953878909), (10, 6, 47.58100112735041), (10, 7, 53.06508026768146), (10, 8, 52.24890709967355), (10, 9, 51.927842577806466), (10, 10, 53.177411550761036), (10, 11, 53.123889205288975), (10, 12, 53.90778745528113), (10, 13, 50.39847105351681), (10, 14, 49.199421670460225), (10, 15, 53.947039040489514), (10, 16, 54.60859618108779), (10, 17, 55.43746722467609), (10, 18, 53.09129191814783), (10, 19, 53.692153471959614), (10, 20, 52.78008359576376), (10, 21, 50.76044343870148), (10, 22, 54.38067905863157), (10, 23, 52.09800015895519), (10, 24, 55.391352068108986), (10, 25, 52.0346149470438), (10, 26, 52.58472571667817), (10, 27, 53.96970888469785), (10, 28, 53.25555034121526), (10, 29, 57.59907599612029), (20, 2, 45.128923405133314), (20, 3, 37.74377175011587), (20, 4, 31.49695684168981), (20, 5, 29.359843697385156), (20, 6, 36.46993666110716), (20, 7, 34.86237817800019), (20, 8, 36.33805437

In [ ]:
%pylab inline
pylab.plot(qual_est.keys(), qual_est.values(), marker='.', label='n_estimators')
pylab.grid(True)
pylab.xlabel('n_estimators')
pylab.ylabel('score')

In [ ]:

pylab.plot(qual_dep.keys(), qual_dep.values(), marker='.', label='max_depth')
pylab.grid(True)
pylab.xlabel('max_depth')
pylab.ylabel('score')

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [ ]:
from sklearn.linear_model import LinearRegression
lrg = LinearRegression()
lrg.fit(X_train, y_train)
print(mean_squared_error(y_test, lrg.predict(X_test))**0.5)